* version check

In [2]:
import torchvision
import torch
print(torch.__version__)
print(torchvision.__version__)

1.9.0+cu102
0.10.0+cu102


In [3]:
!which python

/home/waue0920/anaconda3/bin/python


## import

In [4]:
from Nb_utils import *
import copy
import random
import torch.nn.functional as F

/home/waue0920/.local/lib/python3.8/site-packages/torchvision/transforms/transforms.py:834: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [8]:
    def resize(model, data):
        data = copy.deepcopy(data)
        bs = len(data)
        for i in range(bs):
            img = data[i]['image']
            h, w = img.shape[-2], img.shape[-1]
            ratio = random.uniform(0.5, 1.0)
            d_h, d_w = int(h * ratio), int(w * ratio)
            x1 = int((w - d_w) / 2)
            y1 = int((h - d_h) / 2)
            bg = torch.zeros_like(img)
            try:
                bg += model.pixel_mean.cpu().int()
            except:
                bg += model.module.pixel_mean.cpu().int()
            bg[:, y1:y1 + d_h, x1:x1 + d_w] = F.interpolate(img.unsqueeze(0).float(),
                                                            size=(d_h, d_w),
                                                            align_corners=False,
                                                            mode='bilinear').squeeze(0)
            data[i]['image'] = bg
            if data[i]['instances'].has('gt_boxes'):
                data[i]['instances'].gt_boxes.tensor *= ratio
                data[i]['instances'].gt_boxes.tensor[:, 0] += x1
                data[i]['instances'].gt_boxes.tensor[:, 2] += x1
                data[i]['instances'].gt_boxes.tensor[:, 1] += y1
                data[i]['instances'].gt_boxes.tensor[:, 3] += y1

            if data[i]['instances'].has('pseudo_boxes'):
                data[i]['instances'].pseudo_boxes.tensor *= ratio
                data[i]['instances'].pseudo_boxes.tensor[:, 0] += x1
                data[i]['instances'].pseudo_boxes.tensor[:, 2] += x1
                data[i]['instances'].pseudo_boxes.tensor[:, 1] += y1
                data[i]['instances'].pseudo_boxes.tensor[:, 3] += y1
        return data

## load model

### load local previous

In [9]:
#LOCAL_PREVIOUS:c2b_source-only 
#    "keep_experiments/c2b_source-only/model_final.pth"
local_model_previous_path = "c2b_source-only"
local_previous_model = load_sourceonlyModel(local_model_previous_path,'final')

keep_experiments/c2b_source-only/cfg.yaml
keep_experiments/c2b_source-only/model_final.pth


AssertionError: Config file 'keep_experiments/c2b_source-only/cfg.yaml' does not exist!

### load global

In [10]:
global_model_path = "output/FedAvg_ck2b/FedAvg_0_20230116.pth" 
cfg = setup("keep_experiments/c2b/cfg.yaml")
Trainer =DefaultTrainer
global_model = Trainer.build_model(cfg)    
DetectionCheckpointer(global_model).resume_or_load(global_model_path, resume=False)

AssertionError: Config file 'keep_experiments/c2b/cfg.yaml' does not exist!

* check dataset

In [ ]:
cfg['DATASETS']['TRAIN_LABEL']

### copy global to local

In [2]:
net = copy.deepcopy(global_model)


NameError: name 'copy' is not defined

## load training data

In [3]:
from pt.data.build import build_detection_sourceonly_loader_two_crops
from pt.data.dataset_mapper import DatasetMapperTwoCropSeparate

mapper = DatasetMapperTwoCropSeparate(cfg, True)
train_data_loader = build_detection_sourceonly_loader_two_crops(cfg, mapper)

NameError: name 'cfg' is not defined

In [ ]:
data_loader_iter = iter(train_data_loader)
train_data =  next(data_loader_iter)

In [ ]:
label_data_q, label_data_k = train_data

In [ ]:
len(label_data_q)

In [ ]:
label_data_q.extend(label_data_k)


In [ ]:
import copy
import random
import torch.nn.functional as F

label_data_q = resize(global_model,label_data_q)

## set model mode

In [ ]:
global_model.eval()
local_previous_model.eval()

In [ ]:
#global_model.roi_heads.box_predictor.cls_score.weight

* check model mode

In [ ]:
global_model.training


* make model req_grad=false

In [ ]:
for param in global_model.parameters():
    param.requires_grad = False
#global_w = global_model.state_dict()

In [ ]:
for param in local_previous_model.parameters():
    param.requires_grad = False



## call model and get loss

In [4]:
from detectron2.utils.events import EventStorage
with EventStorage() as storage:
    #record_dict_global, _, _, _ = global_model(label_data_q, branch="supervised")
    #record_dict_local_prev, _, _, _ = local_previous_model(label_data_q, branch="supervised")
    record_dict_local_curr, _, _, _ = net(label_data_q, branch="supervised")

NameError: name 'net' is not defined

In [5]:

A = global_model(label_data_q, branch="unsup_data_weak_test")

NameError: name 'global_model' is not defined

In [6]:
features,proposals_rpn, proposals_roih, ROI_predictions = net(label_data_q, branch="unsup_data_weak_test")

NameError: name 'net' is not defined

In [7]:
with torch.no_grad():
    (_,  proposals_rpn_unsup_k, proposals_roih_unsup_k, _,)= net(label_data_q, branch="unsup_data_weak")

NameError: name 'torch' is not defined

In [8]:
features

NameError: name 'features' is not defined

In [9]:
A[4]

NameError: name 'A' is not defined

In [10]:
global_model.backbone(label_data_q)


NameError: name 'global_model' is not defined

* ### TODO!!! take feature to do similarity ###

In [24]:
cos=torch.nn.CosineSimilarity(dim=-1)


### loss1

In [15]:
record_dict_local_curr

{'loss_cls': tensor(0.0350, device='cuda:0', grad_fn=<MulBackward0>),
 'loss_box_reg': tensor(0.4735, device='cuda:0', grad_fn=<MulBackward0>),
 'loss_rpn_cls': tensor(0.0475, device='cuda:0', grad_fn=<MulBackward0>),
 'loss_rpn_loc': tensor(0.4003, device='cuda:0', grad_fn=<MulBackward0>)}

In [16]:
loss_dict = {}
for key in record_dict_local_curr.keys():
    if key.split('_')[-1] == "adv":
        loss_dict[key] = record_dict_local_curr[key] * 0.0
    elif key[:4] == "loss":
        loss_dict[key] = record_dict_local_curr[key] * 1.0
losses = sum(loss_dict.values())

In [17]:
losses

tensor(0.9563, device='cuda:0', grad_fn=<AddBackward0>)

In [21]:

optimizer = torch.optim.SGD(net.parameters(), lr = 0.01, momentum=0.9)
optimizer.zero_grad()  
losses.backward()
optimizer.step()


TypeError: __init__() missing 1 required positional argument: 'params'

NameError: name 'optimizer' is not defined

In [14]:
record_dict_local_prev

{'loss_cls': tensor(0.0243, device='cuda:0', grad_fn=<MulBackward0>),
 'loss_box_reg': tensor(0.3738, device='cuda:0', grad_fn=<MulBackward0>),
 'loss_rpn_cls': tensor(0.0262, device='cuda:0', grad_fn=<MulBackward0>),
 'loss_rpn_loc': tensor(0.3694, device='cuda:0', grad_fn=<MulBackward0>)}

## appendix

### moon code

In [ ]:
            posi = cos(pro1, pro2)
            logits = posi.reshape(-1,1)

            for previous_net in previous_nets:
                previous_net.cuda()
                _, pro3, _ = previous_net(x)
                nega = cos(pro1, pro3)
                logits = torch.cat((logits, nega.reshape(-1,1)), dim=1)

                previous_net.to('cpu')

            logits /= temperature
            labels = torch.zeros(x.size(0)).cuda().long()

            loss2 = mu * criterion(logits, labels)


            loss1 = criterion(out, target)
            loss = loss1 + loss2

### iteration

In [ ]:

dataset_name = 'VOC2007_citytrain'
train_data_loader = build_train_loader(cfg, dataset_name)
for idx, inputs in enumerate(train_data_loader):    

    record_dict, _, _, _ = global_model(inputs, branch="supervised")

###  call model and get loss

In [ ]:
record_dict, _, _, _ = self.model(label_data_q, branch="supervised")